In [9]:
import pyodbc
# Connect to the database
server = 'DIST-6-505.uopnet.plymouth.ac.uk'
database = 'COMP2001_MFerguson'
username = 'MFerguson'
password = 'GjiF140*'
driver = '{ODBC Driver 17 for SQL Server}'

conn_str = (
    f'DRIVER={driver};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'UID={username};'
    f'PWD={password};'
    'Encrypt=Yes;'
    'TrustServerCertificate=Yes;'
    'Connection Timeout=30;'
    'Trusted_Connection=No;'
)
conn = pyodbc.connect(conn_str)
# Create a cursor from the connectiob, this is used to execute queries
cursor = conn.cursor()

In [7]:
# CW2 schema
cursor.execute('CREATE SCHEMA CW2') 
conn.commit()

ProgrammingError: ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'CW2' in the database. (2714) (SQLExecDirectW); [42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]CREATE SCHEMA failed due to previous errors. (2759)")

In [12]:
conn.close()

ProgrammingError: Attempt to use a closed connection.